In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from hfgtools import *

In [4]:
bgtest = gpd.read_file(path+'\\Homes_for_Good\\Homes_for_Good.gdb', layer='bg_example')

In [7]:
testdf = bgtest.Address.value_counts().rename_axis('address').reset_index(name='counts')

In [8]:
testdf.counts.sum()

1480

In [3]:
state = gpd.read_file(r'T:\DCProjects\Data\Census\Shp\cb_2020_us_state_500k\cb_2020_us_state_500k.shp')

In [4]:
counties = gpd.read_file(path+'\\Homes_for_Good\\Homes_for_Good.gdb', layer='Oregon_Counties')

In [5]:
eugspr = gpd.read_file(r'T:\DCProjects\Support\Lane\HfG\Homes_for_Good\Homes_for_Good.gdb',
                      layer='BG_TitleVI_EugSpr')

In [6]:
oa_gdf = gpd.read_file(path+'\\output\\online_application.shp')

In [7]:
am_gdf = gpd.read_file(path+'\\output\\application_members.shp')

In [8]:
am_gdf.shape

(47804, 40)

In [9]:
amu_gdf = am_gdf.drop_duplicates(subset=['ID'])

In [10]:
amu_gdf.shape

(11354, 40)

In [11]:
state.columns

Index(['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'STUSPS', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry'],
      dtype='object')

In [12]:
polygons=[state, counties, zip_shp, eugspr]

In [13]:
idcols=['STUSPS', 'GEOID', 'ZipCode', 'BlkGrp20']

In [14]:
shpnms=['state', 'county', 'zipcode', 'bg']

In [25]:
for i in range(4):
    polygon = polygons[i]
    idcol = idcols[i]
    out = combine_counts_pip(pntlist=[oa_gdf, amu_gdf], 
                                  polygon=polygon, 
                                  idcol=idcol, 
                                  cntnmlist=['OACNT', 'AMCNT'])
    newpath = path+f'\\output\\{shpnms[i]}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if idcol in ['STUSPS', 'GEOID']:
        out[[idcol, 'NAME', 'OACNT', 'AMCNT', 'geometry']].to_file(newpath+f'\\{shpnms[i]}_counts.shp')
    else:
        out[[x for x in out.columns if len(x)<=10]].to_file(newpath+f'\\{shpnms[i]}_counts.shp')   